In [1]:
import sys
from gapfill_utilities import utils as gf_utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
sys.path.append('/home/blattms1/projects/gapfill/4CL_JAK2/likelihood_model/src/')
import utils as lm_utils

In [2]:
def get_gapfill_likelihoods(gapfill,error_rate_dicts,alleles):
    likelihoods = lm_utils.get_likelihoods_of_true_allele(gapfill, error_rate_dicts)
    likelihood_list = {}
    for allele in alleles:
        if allele in likelihoods:
            likelihood_list[allele] = likelihoods[allele]
        else:
            likelihood_list[allele] = 10e-10
    return likelihood_list
error_rate_dir = '/home/blattms1/projects/gapfill/4CL_JAK2/likelihood_model/error_rate_dicts_HB'
error_rate_dicts = lm_utils.get_error_rate_dicts(error_rate_dir)

In [3]:
alt_gapfill_set1 = pd.read_csv('../part1/likelihood_tables/all_gapfill_features_filtered.csv')
alt_gapfill_set2 = pd.read_csv('../part2/likelihood_tables/all_gapfill_features_filtered.csv')
alt_gapfill_set_combined = pd.concat([alt_gapfill_set1, alt_gapfill_set2])
alt_gapfill_set_combined = alt_gapfill_set_combined.sort_values('likelihood_wt_given_observed').drop_duplicates(subset='name')

alt_gapfill_set_combined.to_csv('likelihood_tables/all_gapfill_features_filtered_combined.csv')

In [4]:
lib = '1'


In [5]:
### first get probe_reads to use for the patient

for BC in ['BC001', 'BC002', 'BC003', 'BC004', 'BC005', 'BC006', 'BC007', 'BC008', 'BC009', 'BC010', 'BC011', 'BC012', 'BC013', 'BC014', 'BC015', 'BC016']:
    print('Processing BC:', BC)
    adata_path = '/data1/lareauc/users/blattms/gapfill/wta_adata/20250605_MPN_' + lib + '_' + BC + '.h5ad'
    gf_dirs = {}
    gf_dirs[0] = '/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_16plex_part' + lib + '_1/BC' + str(int(BC.replace('BC',''))) + '_giftwrap/'
    gf_dirs[1] = '/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_16plex_part' + lib + '_2/BC' + str(int(BC.replace('BC',''))) + '_giftwrap/'
    
    min_percent_supporting = 0.9
    collapse_across_probes = True
    for key,gf_dir in gf_dirs.items():
        if key == 0:
            probe_reads = gf_utils.get_input_probe_reads(gf_dir, read_threshold = 0, cell_barcode_suffix = '-' + str(key), adata_path = adata_path, min_percent_supporting=min_percent_supporting, collapse_across_probes=collapse_across_probes)
        else:
            probe_reads = pd.concat([probe_reads, gf_utils.get_input_probe_reads(gf_dir, read_threshold = 0, cell_barcode_suffix = '-' + str(key), adata_path = adata_path, min_percent_supporting=min_percent_supporting, collapse_across_probes=collapse_across_probes)], ignore_index=True)

    # alt_gapfill_set = pd.read_csv('likelihood_tables/' + BC + '_alt_gapfill_set.csv')
    alt_gapfill_set = pd.read_csv('likelihood_tables/all_gapfill_features_filtered_combined.csv')
    if len(alt_gapfill_set) > 0:
        ## reformat alt gapfill set
        manifest = gf_utils.get_manifest(gf_dirs[0])
        probe_name_to_probe_idx = dict(zip(manifest['name'], manifest['index']))
        alt_gapfill_set['probe_idx'] = alt_gapfill_set['name'].map(probe_name_to_probe_idx)
        
        alt_gapfill_set = pd.concat([
            alt_gapfill_set[['probe_idx', 'name', 'gapfill']].rename(columns={'gapfill': 'gapfill_value'}).assign(variant=True),
            alt_gapfill_set[['probe_idx', 'name', 'gapfill_from_transcriptome']].rename(columns={'gapfill_from_transcriptome': 'gapfill_value'}).assign(variant=False)
        ], ignore_index=True)

        alt_gapfill_set = alt_gapfill_set.dropna(subset='probe_idx')
        # print(alt_gapfill_set)
        
        ### from now on, we only need to consider probes that have an alt gapfill
        probe_reads = probe_reads.loc[probe_reads['probe_idx'].isin(alt_gapfill_set['probe_idx'])]

        unique_gapfill_list = probe_reads[['probe_idx','gapfill']].drop_duplicates()

        possible_alleles = {}
        for probe_idx in probe_reads['probe_idx'].unique():
            possible_alleles[probe_idx] = alt_gapfill_set.loc[alt_gapfill_set['probe_idx'] == probe_idx, 'gapfill_value'].unique()

        unique_gapfill_list['likelihood'] = unique_gapfill_list.apply(
            lambda row: get_gapfill_likelihoods(row['gapfill'], error_rate_dicts, possible_alleles[row['probe_idx']]), axis=1
        )

        likelihood_keys = list(set(key for likelihood in unique_gapfill_list['likelihood'] for key in likelihood.keys()))
        for key in likelihood_keys:
            unique_gapfill_list[key] = unique_gapfill_list['likelihood'].apply(lambda x: x.get(key, None))

        unique_gapfill_list = unique_gapfill_list.drop(columns=['likelihood'])

        unique_gapfill_list.to_csv('likelihood_tables/' + BC + '_gapfill_likelihoods.csv', index=False)

Processing BC: BC001


568648 UMIs found


Collapsing UMIs across probes, 568648 UMIs remaining (100.00%)
Filtering probe reads based on read threshold (0) and min percent supporting (0.9), 566830 UMIs remaining (99.68%)


Filtering cells based on min counts (0) and genes (0) in WTA


Filtering probe reads based on cell barcodes in adata, 553555 UMIs remaining (97.35%)


...

1630738 UMIs found


Collapsing UMIs across probes, 1630738 UMIs remaining (100.00%)


Filtering probe reads based on read threshold (0) and min percent supporting (0.9), 1623116 UMIs remaining (99.53%)


Filtering cells based on min counts (0) and genes (0) in WTA


Filtering probe reads based on cell barcodes in adata, 1550467 UMIs remaining (95.08%)
